### Sequence - to - Sequence

- 인코더와 디코더 라는 두 개의 모듈로 구성되어 있다.
     - 인코더는 입력 문장의 모든 단어들을 순차적으로 입력받은 뒤에 마지막에 이 모든 단어 정보들을 압축해서 하나의 벡터로 만드는데, 이를 context vector라고 한다.
     - 입력 문장의 정보가 하나의 컨텍스트 벡터로 모두 압축되면 인코더는 컨텍스트 벡터를 디코더로 전송, 디코더는 컨텍스트 벡터를 받아서 번역된 단어를 한 개씩 순차적으로 출력 한다.
     


In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model, load_model

In [2]:
df = pd.read_csv('../data/sample/korean_correct_train_data_100000.csv')

In [3]:
len(df)

100000

In [4]:
df = df.iloc[:20000]

In [5]:
df['tgt'] = df.tgt.apply(lambda x: '\t ' + x + ' \n')
print(df['tgt'])

0        \t 증류수로 희석한 탈지유와 수프는 생산자 지침에 표시된 농도의 절반으로 증류수로...
1        \t 다른 한편으로는 파괴력에 대한 합리적인 유연성과 빌딩 블록과 신호 분자가 살아...
2        \t 식품위생감독은 기술과제임과 동시에 법에 따른 행정적 업무로서 감독자는 견실하고...
3        \t 예심에 합격한 자는 시 품질감독부서가 성급 품질감독부서와 품질감독검사검역총국에...
4        \t 케이크 가루는 기포 분포가 매우 균일하며 베이킹 후에도 원래 구조가 여전히 유...
                               ...                        
19995            \t 그물망은 기상 조건이 수분 매개체 활동에 최적일 때 발생합니다. \n
19996    \t 죽림 관리 조치는 죽림 생태계를 교란시키고 토양의 양분을 고갈시키며 토양의 기...
19997    \t 그러나 그 시스템은 한 번도 사용되지 않았고 비축량의 쌀도 실제 비상상황에 대...
19998                    \t 하나의 겔 레인은 은색 염색 시약으로 염색하였다. \n
19999             \t 이 예측을 바탕으로 대장균의 세포질에서 아게리틴을 발현시켰다. \n
Name: tgt, Length: 20000, dtype: object


In [6]:
src_vocab = set()

for line in df['src'].tolist():
    for c in line:
        src_vocab.add(c)

In [7]:
tgt_vocab = set()

for line in df['tgt'].tolist():
    for c in line:
        tgt_vocab.add(c)

In [8]:
src_vocab_size = len(src_vocab) + 1
tgt_vocab_size = len(tgt_vocab) + 1

In [9]:
src_to_index = dict([(word, i+1) for i , word in enumerate(src_vocab)])
tgt_to_index = dict([(word, i+1) for i , word in enumerate(tgt_vocab)])

In [10]:
encoder_input = []

for line in df['src'].tolist():
    encoded_line = []
    for c in line:
        encoded_line.append(src_to_index[c])
    encoder_input.append(encoded_line)

In [11]:
decoder_input = []

for line in df['tgt'].tolist():
    encoded_line = []
    for c in line:
        encoded_line.append(tgt_to_index[c])
    decoder_input.append(encoded_line)

In [12]:
decoder_tgt = []

for line in df['tgt'].tolist():
    ts = 0
    encoded_line = []
    for c in line:
        if ts > 0:
            encoded_line.append(tgt_to_index[c])
        ts += 1
    decoder_tgt.append(encoded_line)

In [13]:
decoder_tgt[0]

[467,
 1141,
 802,
 469,
 1238,
 467,
 1143,
 346,
 1179,
 467,
 434,
 531,
 206,
 1219,
 467,
 469,
 533,
 10,
 467,
 457,
 521,
 880,
 467,
 531,
 246,
 320,
 467,
 149,
 1000,
 656,
 467,
 190,
 140,
 845,
 467,
 649,
 47,
 1013,
 1238,
 467,
 1141,
 802,
 469,
 1238,
 467,
 856,
 404,
 778,
 1195,
 906,
 467,
 253]

In [14]:
max_src_len = max([len(line) for line in df['src'].tolist()])
max_tgt_len = max([len(line) for line in df['tgt'].tolist()])

In [15]:
encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tgt_len, padding='post')
decoder_tgt = pad_sequences(decoder_tgt, maxlen=max_tgt_len, padding='post')


In [16]:
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_tgt = to_categorical(decoder_tgt)

In [17]:
def seq2seq(source_vocab_size, target_vocab_size):
    encoder_inputs = Input(shape=(None, source_vocab_size))
    encoder_lstm = LSTM(units=256, return_state=True)
    
    encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
    # LSTM은 바닐라 RNN과는 달리 상태가 두 개. 은닉 상태와 셀 상태.
    encoder_states = [state_h, state_c]
    
    decoder_inputs = Input(shape=(None, target_vocab_size))
    decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)
    
    # 디코더에게 인코더의 은닉 상태, 셀 상태를 전달.
    decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state=encoder_states)

    decoder_softmax_layer = Dense(target_vocab_size, activation='softmax')
    decoder_outputs = decoder_softmax_layer(decoder_outputs)

    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    model.compile(optimizer="rmsprop", loss="categorical_crossentropy")
    
    return model

In [18]:
seq2seq_model = seq2seq(src_vocab_size,tgt_vocab_size)

2022-10-04 09:19:21.750776: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [19]:
seq2seq_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, 1435)  0           []                               
                                ]                                                                 
                                                                                                  
 input_2 (InputLayer)           [(None, None, 1309)  0           []                               
                                ]                                                                 
                                                                                                  
 lstm (LSTM)                    [(None, 256),        1732608     ['input_1[0][0]']                
                                 (None, 256),                                                 

In [21]:
# 학습 및 모델 저장.
seq2seq_model.fit(x=[encoder_input, decoder_input], y=decoder_tgt, batch_size=64, epochs=5, validation_split=0.2)
seq2seq_model.save('../seq2seq_model.h5')

Epoch 1/5
250/250 [==============================] - 881s 4s/step - loss: 1.3199 - val_loss: 1.0765
Epoch 2/5
250/250 [==============================] - 875s 4s/step - loss: 0.9779 - val_loss: 0.8791
Epoch 3/5
250/250 [==============================] - 1792s 7s/step - loss: 0.8617 - val_loss: 0.7753
Epoch 4/5
250/250 [==============================] - 776s 3s/step - loss: 0.7447 - val_loss: 0.7175
Epoch 5/5
250/250 [==============================] - 869s 3s/step - loss: 0.6996 - val_loss: 0.6785


### 기계번역기 동작

1. 번역하고자 하는 입력 문장이 인코더에 들어가서 은닉상태와 셀 상태를 얻는다.
2. 상태와 '\t'를 디코더로 보낸다.
3. 디코더가 '\n'이 나올때 까지 다음 문자를 예측하는 행동을 반복한다.

In [21]:
model = load_model('../models/seq2seq_model.h5')

In [22]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, None, 1435)  0           []                               
                                ]                                                                 
                                                                                                  
 input_4 (InputLayer)           [(None, None, 1309)  0           []                               
                                ]                                                                 
                                                                                                  
 lstm_2 (LSTM)                  [(None, 256),        1732608     ['input_3[0][0]']                
                                 (None, 256),                                                 

In [33]:
encoder_inputs = Input(shape=(None, src_vocab_size))
encoder_lstm = LSTM(units=256, return_state=True)

encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
# LSTM은 바닐라 RNN과는 달리 상태가 두 개. 은닉 상태와 셀 상태.
encoder_states = [state_h, state_c]

encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

decoder_inputs = Input(shape=(None, tgt_vocab_size))
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)

# 디코더에게 인코더의 은닉 상태, 셀 상태를 전달.
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state=encoder_states)

decoder_softmax_layer = Dense(tgt_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

In [34]:
model.fit(x=[encoder_input, decoder_input], y=decoder_tgt, batch_size=64, epochs=5, validation_split=0.2)
model.save('../seq2seq_model_2.h5')

Epoch 1/5
250/250 [==============================] - 805s 3s/step - loss: 1.3254 - val_loss: 1.0580
Epoch 2/5
250/250 [==============================] - 1391s 6s/step - loss: 0.9839 - val_loss: 0.8725
Epoch 3/5
250/250 [==============================] - 845s 3s/step - loss: 0.8222 - val_loss: 0.7662
Epoch 4/5
250/250 [==============================] - 862s 3s/step - loss: 0.7366 - val_loss: 0.7090
Epoch 5/5
250/250 [==============================] - 924s 4s/step - loss: 0.6911 - val_loss: 0.6744


In [35]:
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용.
# 뒤의 함수 decode_sequence()에 동작을 구현 예정
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_softmax_layer = Dense(tgt_vocab_size, activation='softmax')

# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태를 버리지 않음.
decoder_states = [state_h, state_c]

decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)

In [36]:
# 단어로부터 인덱스를 얻는 것이 아닌 인덱스로부터 단어를 얻을 수 있는 index_to_src, index_to_tgt 를 만듦
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tgt = dict((i, char) for char, i in tgt_to_index.items())

In [37]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, tgt_vocab_size))
    target_seq[0, 0, tgt_to_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ""
    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
    # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tgt[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '\n' or
            len(decoded_sentence) > max_tgt_len):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1, tgt_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1.

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [38]:
for seq_index in [1,2,20]: # 입력 문장의 인덱스
    input_seq = encoder_input[seq_index:seq_index+1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', df.src[seq_index])
    print('정답 문장:', df.tgt[seq_index][2:len(df.tgt[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
    print('번역 문장:', decoded_sentence[1:len(decoded_sentence)-1]) # '\n'을 빼고 출력

1/1 [==============================] - 0s 14ms/step


1/1 [==============================] - 0s 15ms/step
-----------------------------------
입력 문장: 다른 한펴느로는 파괴려게 대한 함니저기 뉴연성과 빌딩 블록꽈 신호 분자가 사라인는 세포에 드러갈 쑤 읻또록 충분한 투과성으 류지해야 한다.
정답 문장: 다른 한편으로는 파괴력에 대한 합리적인 유연성과 빌딩 블록과 신호 분자가 살아있는 세포에 들어갈 수 있도록 충분한 투과성을 유지해야 한다. 
번역 문장: 행슬놈왔텝엇렀렀렀렀렀렀렀렀렀렀렀렀렀렀핵핵핵핵핵핵핵핵핵핵핵핵핵핵핵핵핵핵핵핵핵혐뒤렀핵핵핵핵핵핵핵핵핵혐렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀
1/1 [==============================] - 0s 14ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 14ms/step
-----------------------------------
입력 문장: 식푸뮈생감도근 기술과제임과 동시에 버베 따른 행정저 검무로서 감독짜는 견실하고 유연한 기술지식 끼바늘 갇추어야 하며 범뉴레 대한 기초지식꽈 행정에 대한 자겨글 갇춘 정치적 짜지를 갇추어야 한다.
정답 문장: 식품위생감독은 기술과제임과 동시에 법에 따른 행정적 업무로서 감독자는 견실하고 유연한 기술지식 기반을 갖추어야 하며 법률에 대한 기초지식과 행정에 대한 자격을 갖춘 정치적 자질을 갖추어야 한다. 
번역 문장: 행슬놈왔텝엇렀렀렀렀렀렀렀렀렀렀렀렀렀렀핵핵핵핵핵핵핵핵핵핵핵핵핵핵핵핵핵핵핵핵핵혐뒤렀핵핵핵핵핵핵핵핵핵혐렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀
1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 14ms/step
-----------------------------------
입력 문장: 그러나 디테르펜 화함무른 커피 종이로 걸러낼 쑤 이쓰므로 여과되지 아는 커피나 금송망으로 여과된 커피는 마시지 안는 거시 조씀니다.
정답 문장: 그러나 디테르펜 화합물은 커피 종이로 걸러낼 수 있으므로 여과되지 않은 커피나 금속망으로 여과된 커피는 마시지 않는 것이 좋습니다. 
번역 문장: 행슬놈왔텝엇렀렀렀렀렀렀렀렀렀렀렀렀렀렀핵핵핵핵핵핵핵핵핵핵핵핵핵핵핵핵핵핵핵핵핵혐뒤렀핵핵핵핵핵핵핵핵핵혐렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀렀
